In [ ]:
text='yeah,but no,but yeah,but no,but yeah'
text.replace('yeah','yep')
text='Today is 11/27/2012. PyCon starts 3/13/2013.'
import re
re.sub(r'(\d+)/(\d+)/(\d+)',r'\3-\1-\2',text)


In [ ]:
#不区分大小写
import re
text='UPPER PYTHON, lower python, Mixed Python'
re.findall('python',text,flags=re.IGNORECASE)
re.sub('python','snake',text,flags=re.IGNORECASE)

In [ ]:
text2='Computer says "no." Phone says "yes."'
str_pat1=re.compile(r'\"(.*)\"')
#贪婪匹配
str_pat1.findall(text2)
#最短匹配
str_pat2=re.compile(r'\"(.*?)\"')
str_pat2.findall(text2)


In [ ]:
s='pýtĥöñ\fis\tawesome\r\n'
remap={
    ord('\t'):' ',
    ord('\f'):' ',
    ord('\r'):None
}
a=s.translate(remap)
a

In [ ]:
text='Hello World'
text.ljust(20,'=')
text.center(20,'$')

In [ ]:
parts=['Is', 'Chicago', 'Not', 'Chicago?']
' '.join(parts)
','.join(parts)
''.join(parts)
data=['ACME', 50, 91.1]
','.join(str(d) for d in data)

In [ ]:
import os
def sample():
    yield 'Is'
    yield 'Chicago'
    yield 'Not'
    yield 'Chicago'
text=''.join(sample())
print(text)
os.chdir(r'C:\Users\xxcmt\Desktop\文档\automate_online-materials')
with open ('join.txt','w') as f:
    for part in sample():
        f.write(part)
def combine(source,maxsize):
    parts=[]
    size=0
    for part in source:
        parts.append(part)
        size+=len(part)
        print(size)
        if size>maxsize:
            yield ''.join(parts)
            parts=[]
            size=0
    yield ''.join(parts)
with open('size.txt','w') as f:
    for part in combine(sample(),32768):
        f.write(part)

In [ ]:
class safesub(dict):
    def __missing__(self,key):
        return '{'+key+'}'
s='{name} has {n} messages.'
name='Guido'
n=37
del n
name='xiong'
#s.format_map(safesub(vars()))
s.format(name='cmt',n=33)

In [ ]:
s="Look into my eyes, look into my eyes, the eyes, the eyes,  the eyes, not around the eyes, don't look around the eyes,  look into my eyes, you're under."
import textwrap
print(textwrap.fill(s,70))
print('#'*35)
print(textwrap.fill(s,40))
print('#'*35)
print(textwrap.fill(s,40,initial_indent='    '))
print('#'*35)
print(textwrap.fill(s,40,subsequent_indent='    '))
print('#'*35)

In [ ]:
#还有这种骚操作
text='foo=23+42*10'
tokens=[('NAME','foo'),('EQ','='),('NUM','23'),('PLUS','+'),
       ('NUM','42'),('TIMES','*'),('NUM','10')]
import re
NAME=r'(?P<NAME>[a-zA-Z_][a-zA-Z_0-9]*)'
NUM=r'(?P<NUM>\d+)'
PLUS=r'(?P<PLUS>\+)'
TIMES=r'(?P<TIMES>\*)'
EQ=r'(?P<EQ>=)'
WS=r'(?P<WS>\s+)'
master_pat=re.compile('|'.join([NAME,NUM,PLUS,TIMES,EQ,WS]))
scanner=master_pat.scanner('foo= 42')
m=scanner.match()
m.lastgroup,m.group()
m=scanner.match()
m.lastgroup,m.group()
m=scanner.match()
m.lastgroup,m.group()
#m_.lastgroup,m_.group()
#scanner.match()
#_.lastgroup,_.group()

In [ ]:
from collections import namedtuple
import re
NAME=r'(?P<NAME>[a-zA-Z_][a-zA-Z_0-9]*)'
NUM=r'(?P<NUM>\d+)'
PLUS=r'(?P<PLUS>\+)'
TIMES=r'(?P<TIMES>\*)'
EQ=r'(?P<EQ>=)'
WS=r'(?P<WS>\s+)'
master_pat=re.compile('|'.join([NAME,NUM,PLUS,TIMES,EQ,WS]))
Token=namedtuple('Token',['type','value'])
def generate_tokens(pat,text):
    scanner=pat.scanner(text)
    for m in iter(scanner.match,None):
        yield Token(m.lastgroup,m.group())
        
for tok in generate_tokens(master_pat,'foo = 42'):
    print(tok)
    
tokens=(tok for tok in generate_tokens(master_pat,text)
       if tok.type!='WS')
for tok in tokens:
    print(tok)

In [ ]:
import re
from collections import namedtuple
NAME=r'(?P<NAME>[a-zA-Z_][a-zA-Z_0-9]*)'
PRINT=r'(?P<PRINT>print)'
Token=namedtuple('Token',['type','value'])
def generate_tokens(pat,text):
    scanner=pat.scanner(text)
    for m in iter(scanner.match,None):
        yield Token(m.lastgroup,m.group())
master_pat=re.compile('|'.join([NAME,PRINT]))
for tok in generate_tokens(master_pat,'printer'):
    print(tok)

In [2]:
#不看了太难了，这不只是写程序，主要是算法理解不了
import re
import collections
import pdb
pdb.set_trace()
NUM=r'(?P<NUM>\d+)'
PLUS=r'(?P<PLUS>\+)'
MINUS=r'(?P<MINUS>-)'
TIMES=r'(?P<TIMES>\*)'
DIVIDE=r'(?P<DIVIDE>/)'
LPAREN=r'(?P<LPAREN>\()'
RPAREN=r'(?P<RPAREN>\))'
WS=r'(?P<WS>\s+)'
master_pat=re.compile('|'.join([NUM,PLUS,MINUS,TIMES,DIVIDE,LPAREN,RPAREN,WS]))
Token=collections.namedtuple('Token',['type','value'])
def generate_tokens(text):
    scanner=master_pat.scanner(text)
    #这里其实还有一点超前iter
    for m in iter(scanner.match,None):
        tok=Token(m.lastgroup,m.group())
        if tok.type!='WS':
            yield tok
class ExpressionEvaluator:
    def parse(self,text):
        self.tokens=generate_tokens(text)
        self.tok=None
        self.nexttok=None
        self._advance()
        return self.expr()
    
    def _advance(self):
        self.tok,self.nexttok=self.nexttok,next(self.tokens,None)
        
    def _accept(self,toktype):
        if self.nexttok and self.nexttok.type==toktype:
            self._advance()
            return True
        else:
            return False
    
    def _expect(self,toktype):
        if not self._accept(toktype):
            raise SyntaxError('Expected'+toktype)
            
    def expr(self):
        exprval=self.term()
        while self._accept('PLUS') or self._accept('MINUS'):
            op=self.tok.type
            right=self.term()
            if op=='PLUS':
                exprval+=right
            elif op=='MINUS':
                exprval-=right
        return exprval
    
    def term(self):
        termval=self.factor()
        while self._accept('TIMES') or self._accept('DIVIDE'):
            op=self.tok.type
            right=self.factor()
            if op=='TIMES':
                termval*=right
            elif op=='DIVIDE':
                termval/=right
        return termval
    
    def factor(self):
        if self._accept('NUM'):
            return int(self.tok.value)
        elif self._accept('LPAREN'):
            exprval=self.expr()
            self._expect('RPAREN')
            return exprval
        else:
            raise SyntaxError('Expected NUMBER or LPAREN')

e=ExpressionEvaluator()
e.parse('2+3')
e.parse('2+3*6')

--Return--
> <ipython-input-2-1edd72da1c00>(4)<module>()->None
-> pdb.set_trace()
(Pdb) c


20

In [21]:
x=1234.56789
format(x,'0.2f')
format(x,'>20.1f')
format(x,'<10.1f')
format(x,'^10.1f')
format(x,',.3f')
format(x,'e')
format(x,'0.2e')
print('The value is {:0,.2f}'.format(x))

The value is 1,234.57


In [27]:
x=1234
bin(x)
oct(x)
hex(x)
format(x,'b')
format(x,'o')
format(x,'x')

'4d2'